# 🍽️ Building a hybrid vector search with BGE-M3 and restaurant data

This notebook demonstrates how to build a hybrid vector search system using both dense and sparse embeddings (LaBSE + BGE-M3). Hybrid search combines semantic dense vectors with sparse/full-text signals (e.g., BM25) to improve retrieval quality.

### What you will learn
- Preparing restaurant data for hybrid search
- Creating embedding functions (LaBSE for dense, BGE-M3 for dense + sparse)
- Defining a Milvus collection with hybrid fields (dense + sparse)
- Tokenization and BM25 sparse-function setup for full-text ranking
- Ingesting data and running hybrid searches (weighted rankers)

### Requirements
- `pymilvus[model]`
- `sentence-transformers`
- `pythainlp`
- `pandas`

Run order: Execute cells top-to-bottom. Adjust the connection and credential cells before running cloud-based searches.

Reference: For a step-by-step blog post, see https://wiphoo.dev (search for "hybrid vector search Milvus").

🔎 Step 0 — Table of contents

- step 1 🔌 connect to Milvus
- step 2 🧠 create embedding functions (LaBSE, BGE-M3)
- step 3 🗂️ define collection schema
- step 4 🧭 create collection & indexes
- step 5 🧹 preprocess data (tokenize)
- step 6 📥 generate embeddings (dense + sparse) & insert
- step 7 📊 collection stats & load
- step 8 🔎 search helpers and examples
- step 9 🔀 hybrid search examples (LaBSE + BM25, BGE-M3 dense + sparse)

Run the notebook top-to-bottom. Use the `search_*` helper functions for quick queries.

In [1]:
%pip install -q --upgrade "pymilvus[model]" sentence-transformers pythainlp pandas

Note: you may need to restart the kernel to use updated packages.


🔌 Step 1 — Connect to Milvus

Establish a connection to your Milvus instance (local or managed cloud). Update the URI or token in this cell before running other cells.

Tips:
- For local testing use `http://localhost:19530`.
- For managed cloud, use your cloud URI and token. Verify connectivity before indexing.

In [2]:
# Connect to Milvus
from pymilvus import MilvusClient

# Connect to local Milvus server
client = MilvusClient(uri="http://localhost:19530")

# For Zilliz Cloud, uncomment and fill in your credentials:
# client = MilvusClient(
#     uri="https://<your-endpoint>",
#     token="<your-token>"
# )

Connect to Milvus server (local or cloud).

🧠 Step 2 — Create Embedding Functions

Set up embedding functions for both dense and hybrid models. We'll use LaBSE for multilingual dense embeddings and BGE-M3 for hybrid (dense + sparse) embeddings.

Configuration notes:
- Choose `device` (cpu / cuda) and `batch_size` appropriate for your hardware.
- When using cosine similarity, normalize embeddings if recommended by the model.

🧠 Step 2.1 — LaBSE (Dense Embeddings)

LaBSE is a multilingual sentence transformer suitable for semantic search across Thai and English texts. Use it for dense vector matching and semantic ranking.

In [3]:
# Create embedding functions
from pymilvus.model.dense import SentenceTransformerEmbeddingFunction

# LaBSE: Multilingual dense embedding (good for Thai/English)
labse_embedding_func = SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/LaBSE",
    batch_size=32,
    device="cpu",
    normalize_embeddings=True,  # Recommended for COSINE metric
)

Set up LaBSE embedding (multilingual dense vectors).

🧠 Step 2.2 — BGE-M3 (Hybrid: Dense + Sparse)

BGE-M3 provides both dense vectors and a sparse/text-like representation. We use both outputs to support hybrid ranking (e.g., combine dense semantic scores with BM25-like sparse signals).

In [4]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

bge_m3_embedding_func = BGEM3EmbeddingFunction(
    model_name='BAAI/bge-m3',
    device='cpu',
    use_fp16=False,
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Set up BGE-M3 embedding (dense + sparse for hybrid search).

In [5]:
from pymilvus import DataType

# Define collection schema
collection_name = "restaurants"
if collection_name in client.list_collections():
    client.drop_collection(collection_name)

schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=False,
)

# Main fields
schema.add_field(field_name="id", datatype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=128)
schema.add_field(field_name="title", datatype=DataType.VARCHAR, max_length=512)
schema.add_field(field_name="latitude", datatype=DataType.FLOAT)
schema.add_field(field_name="longitude", datatype=DataType.FLOAT)

# Text for full-text search
schema.add_field(field_name="text_tokenize", datatype=DataType.VARCHAR, max_length=4096,
                enable_analyzer=True, analyzer_params={"tokenizer": "whitespace"})

# Embedding fields
schema.add_field(field_name="labse_dense_vector", datatype=DataType.FLOAT_VECTOR, dim=labse_embedding_func.dim)
schema.add_field(field_name="pythainlp_sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR)
schema.add_field(field_name="bge_m3_dense_vector", datatype=DataType.FLOAT_VECTOR, dim=bge_m3_embedding_func.dim["dense"])
schema.add_field(field_name="bge_m3_sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR)

# Partition key for geo search
schema.add_field(field_name="h3_r8", datatype=DataType.VARCHAR, max_length=32, is_partition_key=True)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 128}, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'latitude', 'description': '', 'type': <DataType.FLOAT: 10>}, {'name': 'longitude', 'description': '', 'type': <DataType.FLOAT: 10>}, {'name': 'text_tokenize', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 4096, 'enable_analyzer': True, 'analyzer_params': '{"tokenizer":"whitespace"}'}}, {'name': 'labse_dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}, {'name': 'pythainlp_sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}, {'name': 'bge_m3_dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'bge_m3_sparse_vector', 'description': '', 'type'

🗂️ Step 3 — Define Collection Schema

Define the collection schema including primary keys, dense vector fields, sparse vector fields, and any metadata (latitude/longitude, partition keys). Keep field names descriptive to simplify queries and output formatting.

🗂️ Step 3 (recap)

Drop existing collection if present, then create a new schema that contains the dense and sparse fields plus optional partition keys for geo-search (e.g., `h3_r8`).

In [6]:
# Add BM25 function (PyThaiNLP)
from pymilvus import Function, FunctionType

# Add BM25 function for full-text search (PyThaiNLP tokenizer)
schema.add_function(Function(
    name="bm25_pythainlp",
    function_type=FunctionType.BM25,
    input_field_names=["text_tokenize"],
    output_field_names=["pythainlp_sparse_vector"],
))

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 128}, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'latitude', 'description': '', 'type': <DataType.FLOAT: 10>}, {'name': 'longitude', 'description': '', 'type': <DataType.FLOAT: 10>}, {'name': 'text_tokenize', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 4096, 'enable_analyzer': True, 'analyzer_params': '{"tokenizer":"whitespace"}'}}, {'name': 'labse_dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}, {'name': 'pythainlp_sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>, 'is_function_output': True}, {'name': 'bge_m3_dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'bge_m3_sparse_vector

Add BM25 function for full-text search (PyThaiNLP).

In [7]:
# Create the collection
client.create_collection(
    collection_name=collection_name,
    schema=schema,
)

Create the collection in Milvus.

In [8]:
# Create indexes for all fields
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="labse_dense_vector",
    index_type="AUTOINDEX",
    metric_type="COSINE",
)
index_params.add_index(
    field_name="pythainlp_sparse_vector",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="BM25",
    params={
        "inverted_index_algo": "DAAT_MAXSCORE",
        "bm25_k1": 1.2,
        "bm25_b": 0.75,
    },
)
index_params.add_index(
    field_name="bge_m3_dense_vector",
    index_type="AUTOINDEX",
    metric_type="IP",
)
index_params.add_index(
    field_name="bge_m3_sparse_vector",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="IP",
)
index_params.add_index(
    field_name="h3_r8",
    index_type="AUTOINDEX",
)

Create indexes for all search fields.

🧭 Step 4 — Create Collection & Indexes

Create the Milvus collection and build indexes for dense and sparse fields. For sparse fields use BM25 inverted index params; for dense vectors use COSINE or IP with AutoIndex/HNSW depending on your scale.

In [9]:
# Build all indexes
client.create_index(collection_name, index_params)

Build all indexes.

In [10]:
# Tokenize text for full-text search (using PyThaiNLP)
from pythainlp.tokenize import word_tokenize

stopwords = ['ร้าน', 'อาหาร', 'สาขา']

def tokenize_and_filter(text: str) -> str:
    tokens = word_tokenize(text, engine="newmm")
    return " ".join([t for t in tokens if t not in stopwords and t.strip() != ""])

Tokenize text for BM25 search (remove stopwords).

---

🔤 Step 5 — Preprocess data (tokenize)

Tokenize and clean restaurant text prior to embedding or BM25 indexing. We use PyThaiNLP for Thai tokenization.

In [11]:
# Load and preprocess restaurant data
import pandas as pd

# load sample restaurant data

df = pd.read_csv("../../data/2025/restaurants/sample_restaurants.csv")
df["combined_text"] = df[["title", "type_ids"]].agg(" ".join, axis=1).str.replace("[", "", regex=False).str.replace("]", "", regex=False).str.replace("\"", "", regex=False)
df["tokenized_separate_by_whitespace"] = df["combined_text"].astype(str).apply(tokenize_and_filter)
df.head()

,title,place_id,latitude,longitude,rating,reviews,types,type_ids,h3_r6,h3_r8,h3_r10,h3_r12,combined_text,tokenized_separate_by_whitespace
0,Catory Pizza สาขาประชาอุทิศ,ChIJP5SViq6j4jAR6C9tANeGbcM,13.652919,100.496844,4.9,936.0,"[""Pizza restaurant""]","[""pizza_restaurant""]",8664a4b27ffffff,8864a4b23dfffff,8a64a4b23c57fff,8c64a4b23cec9ff,Catory Pizza สาขาประชาอุทิศ pizza_restaurant,Catory Pizza ประชา อุทิศ pizza _restaurant
1,ERA VALLEY,ChIJS_B0uzKj4jARyoRkKkXMRo4,13.657025,100.468720,4.6,53.0,"[""Restaurant""]","[""restaurant""]",8664a4b2fffffff,8864a4b2e5fffff,8a64a4b2e4cffff,8c64a4b2e4c13ff,ERA VALLEY restaurant,ERA VALLEY restaurant
2,HUANG หวง เกี๊ยวจีน 黄餃子館,ChIJGyEeYxSZ4jAREqdmdTa_mQ4,13.658768,100.468625,4.9,83.0,"[""Chinese restaurant"",""Chinese takeaway""]","[""chinese_restaurant"",""chinese_takeaway""]",8664a4b2fffffff,8864a4b05bfffff,8a64a4b2e4dffff,8c64a4b2e4db5ff,"HUANG หวง เกี๊ยวจีน 黄餃子館 chinese_restaurant,ch...","HUANG หวง เกี๊ยว จีน 黄餃子館 chinese _restaurant,..."
3,เนื้อเทพ NueaThep (สาขาประชาอุทิศ),ChIJX-tIU6aj4jARBc4SLkbXKM8,13.651996,100.497408,4.7,29.0,"[""Restaurant"",""Noodle shop""]","[""restaurant"",""noodle_shop""]",8664a4b27ffffff,8864a4b23dfffff,8a64a4b23c57fff,8c64a4b23c541ff,"เนื้อเทพ NueaThep (สาขาประชาอุทิศ) restaurant,...",เนื้อ เทพ NueaThep ( ประชา อุทิศ ) restaurant ...
4,Indian Barbeque Nation 57,ChIJBUSnKQqf4jAR_9kF8Wjqwu8,13.645877,100.498482,4.6,43.0,"[""Restaurant""]","[""restaurant""]",8664a4b27ffffff,8864a4b239fffff,8a64a4b23997fff,8c64a4b239959ff,Indian Barbeque Nation 57 restaurant,Indian Barbeque Nation 57 restaurant


Load and preprocess restaurant data.

In [12]:
# Generate embeddings for all documents
bge_m3_embedded_text = bge_m3_embedding_func.encode_documents(df["combined_text"].tolist())
labse_embedded_text = labse_embedding_func.encode_documents(df["combined_text"].tolist())

pre tokenize: 100%|██████████| 17/17 [00:00<00:00, 539.33it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
pre tokenize: 100%|██████████| 17/17 [00:00<00:00, 539.33it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 17/17 [00:19<00:00,  1.14s/it]



Generate dense embeddings for all records.

---

🔢 Step 6 — Preprocess data (generate embeddings)

Use LaBSE and BGE-M3 embedding functions to produce dense vectors and BGE-M3's sparse outputs.

In [13]:
# Convert sparse output to Milvus dict
import numpy as np
bge_m3_embedded_sparse_csr = bge_m3_embedded_text["sparse"].tocsr()
bge_m3_embedded_sparse_csr.sum_duplicates()
bge_m3_embedded_sparse_csr.sort_indices()

def bge_m3_embedded_sparse_row_to_dict(row):
    s, e = bge_m3_embedded_sparse_csr.indptr[row], bge_m3_embedded_sparse_csr.indptr[row + 1]
    idx = bge_m3_embedded_sparse_csr.indices[s:e]
    val = bge_m3_embedded_sparse_csr.data[s:e].astype(np.float32, copy=False)
    return {int(i): float(v) for i, v in zip(idx, val)}

Convert BGE-M3 sparse output to Milvus format.

In [14]:
# Insert data
entities = [
    {
        "id": str(row["place_id"]),
        "title": str(row["title"]),
        "latitude": float(row['latitude']),
        "longitude": float(row['longitude']),
        "text_tokenize": str(row["tokenized_separate_by_whitespace"]),
        "labse_dense_vector": labse_embedded_text[idx],
        "bge_m3_dense_vector": bge_m3_embedded_text["dense"][idx].tolist(),
        "bge_m3_sparse_vector": bge_m3_embedded_sparse_row_to_dict(idx),
        "h3_r8": str(row["h3_r8"]),
    }
    for idx, row in df.iterrows()
]

client.insert(collection_name, entities)
client.flush(collection_name)
client.load_collection(collection_name)
print(f"Inserted {len(entities)} records with embeddings.")

Inserted 268 records with embeddings.


Insert all data and embeddings into Milvus. (Step 6.3)

---

🔎 Step 7 — Collection stats

Check collection statistics and ensure data is loaded for search.

In [15]:
# Check collection stats
print(f"Number of {client.get_collection_stats(collection_name)['row_count']} records in collection {collection_name}.")

Number of 268 records in collection restaurants.


Show number of records in the collection.

---

⚙️ Step 8 — Helpers (to_dataframe, search functions)

Utility helpers for result formatting and search (e.g., `to_dataframe`).

In [16]:
# Helper: Convert Milvus results to DataFrame for easy viewing
from itertools import chain
import pandas as pd

def to_dataframe(data):
    """
    Accepts either a list of dicts or a list of lists of dicts (like your example)
    and returns a flattened DataFrame with `entity_*` columns.
    """
    if data and isinstance(data[0], list):
        records = list(chain.from_iterable(data))
    else:
        records = data
    flat = []
    for item in records:
        base = {k: v for k, v in item.items() if k != "entity"}
        entity = item.get("entity") or {}
        base.update({f"{k}": v for k, v in entity.items()})
        flat.append(base)
    df = pd.DataFrame(flat)
    preferred = ["id", "distance", "title", "latitude", "longitude", "h3_r8"]
    df = df.reindex(columns=[c for c in preferred if c in df.columns] + [c for c in df.columns if c not in preferred])
    return df

Convert Milvus results to DataFrame. (Step 8 helper)

In [17]:
# Search helpers (search_labse, search_fulltext, search_bge_dense, search_bge_sparse)
def search_labse(query: str, limit: int = 5):
    """Encode query with LaBSE and search the labse_dense_vector field."""
    q_emb = labse_embedding_func.encode_queries([query])
    res = client.search(
        collection_name=collection_name,
        data=q_emb,
        anns_field="labse_dense_vector",
        limit=limit,
        search_params={"metric_type": "COSINE", "params": {"nprobe": 10}},
        output_fields=["title", "latitude", "longitude", "h3_r8"],
    )
    return to_dataframe(res)


def search_fulltext(query: str, limit: int = 10):
    """Tokenize query and search the PyThaiNLP BM25 sparse vector field."""
    tokens = tokenize_and_filter(query).split()
    res = client.search(
        collection_name=collection_name,
        data=tokens,
        anns_field="pythainlp_sparse_vector",
        limit=limit,
        search_params={"metric_type": "BM25", "topk": limit},
        output_fields=["id", "title", "latitude", "longitude", "h3_r8"],
    )
    return to_dataframe(res)


def search_bge_dense(query: str, limit: int = 5):
    """Encode query with BGE-M3 and search its dense field."""
    q_emb = bge_m3_embedding_func.encode_queries([query])["dense"]
    res = client.search(
        collection_name=collection_name,
        data=q_emb,
        anns_field="bge_m3_dense_vector",
        limit=limit,
        search_params={"metric_type": "IP", "params": {"nprobe": 10}},
        output_fields=["id", "title", "latitude", "longitude", "h3_r8"],
    )
    return to_dataframe(res)


def search_bge_sparse(query: str, limit: int = 5):
    """Encode query with BGE-M3 and search its sparse field."""
    q_emb = bge_m3_embedding_func.encode_queries([query])["sparse"]
    res = client.search(
        collection_name=collection_name,
        data=q_emb,
        anns_field="bge_m3_sparse_vector",
        limit=limit,
        search_params={"metric_type": "IP", "params": {"nprobe": 10}},
        output_fields=["id", "title", "latitude", "longitude", "h3_r8"],
    )
    return to_dataframe(res)


Helper functions for all search types.

📊 Step 9 — Comparison & Visualization

Run the same query across LaBSE (dense), Full-text (BM25), BGE-M3 dense, and BGE-M3 sparse. Collect and display results side-by-side to compare ranking behavior and retrieved documents. Use the helper functions to normalize and merge scores for hybrid ranking.

🔹 Step 9.1 — LaBSE dense search example

Use LaBSE to encode the query and run a dense vector search. This demonstrates semantic matching using multilingual dense embeddings.

In [18]:
# LaBSE dense search example
query = "ซูชิ"
search_labse(query, limit=5)

,id,distance,title,latitude,longitude,h3_r8
0,ChIJG5cMlYmf4jARjI14Tvhzj5I,0.539514,Sushi Sora,13.726238,100.543182,8864a4b14dfffff
1,ChIJFepMlimf4jARW2MqZCN7GMQ,0.521155,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff
2,ChIJC_IG4WGZ4jARJil71QyJnJQ,0.475555,Min Sushi by Sushi Cottage ずしコテージ,13.740359,100.525108,8864a4b10dfffff
3,ChIJ89mnRNGj4jARXFw7F8kdlu8,0.463266,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
4,ChIJT-MmY-mj4jARBxgvjap0hf0,0.424365,Suki Teenoi Susco Phuttha Bucha,13.651340,100.488991,8864a4b231fffff


LaBSE dense search example.

🔹 Step 9.2 — Full-text (BM25) search example

Tokenize the query and run a BM25 search against the sparse field to demonstrate keyword-style ranking.

In [19]:
# Full-text search example
query_text_tokenized = tokenize_and_filter(query).split()
query_text_tokenized

['ซูชิ']

Tokenize query for BM25 search.

In [20]:
search_fulltext(query, limit=10)

,id,distance,title,latitude,longitude,h3_r8
0,ChIJ89mnRNGj4jARXFw7F8kdlu8,4.527886,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
1,ChIJFepMlimf4jARW2MqZCN7GMQ,4.527886,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff


BM25 full-text search example.

🔹 Step 9.3 — BGE-M3 dense search example

Use BGE-M3 dense vectors for semantic ranking focused on the model's dense representation.

In [21]:
search_bge_dense(query, limit=3)

,id,distance,title,latitude,longitude,h3_r8
0,ChIJ89mnRNGj4jARXFw7F8kdlu8,0.609167,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
1,ChIJFepMlimf4jARW2MqZCN7GMQ,0.597810,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff
2,ChIJG5cMlYmf4jARjI14Tvhzj5I,0.567272,Sushi Sora,13.726238,100.543182,8864a4b14dfffff


BGE-M3 dense search example.

🔹 Step 9.4 — BGE-M3 sparse search example

Use BGE-M3's sparse output (BM25-like) to run sparse searches and compare results with dense-only queries.

In [22]:
search_bge_sparse(query, limit=3)

,id,distance,title,latitude,longitude,h3_r8
0,ChIJFepMlimf4jARW2MqZCN7GMQ,0.095425,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff
1,ChIJ89mnRNGj4jARXFw7F8kdlu8,0.060696,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
2,ChIJ7crcpPaj4jAR3QlRUKokiWI,0.000433,KFC @Susco square,13.650912,100.488785,8864a4b231fffff


BGE-M3 sparse search example.

🔹 Step 9.5 — Hybrid Search Example: LaBSE + BM25

Combine LaBSE dense vector results with BM25 sparse results, normalize scores, and rank by a weighted sum to produce hybrid results that leverage semantic and keyword signals.

In [23]:
# Hybrid search: combine LaBSE dense and BM25 full-text using client.hybrid_search (float fix)
from pymilvus import AnnSearchRequest, WeightedRanker

def search_labse_and_pythainlp(query:str, dense_weight:str=0.5, sparse_weight=0.5):
    # Prepare LaBSE dense query (should be a list of list of float)
    labse_query = labse_embedding_func.encode_queries([query])  # returns [[...]]
    labse_request = AnnSearchRequest(
        data=labse_query,  # [[float, float, ...]]
        anns_field="labse_dense_vector",
        param={
            "metric_type": "COSINE",
            "params": {"nprobe": 10}
        },
        limit=10
    )

    # Prepare BM25 full-text query (data should be list of str)
    fulltext_tokens = [str(t) for t in tokenize_and_filter(query).split()]
    fulltext_request = AnnSearchRequest(
        data=fulltext_tokens,  # [str, str, ...]
        anns_field="pythainlp_sparse_vector",
        param={
            "metric_type": "BM25"
        },
        limit=10
    )

    # Weighted ranker: weights must match number of requests
    ranker = WeightedRanker(dense_weight, sparse_weight)

    # Run hybrid search
    return client.hybrid_search(
        collection_name=collection_name,
        reqs=[labse_request, fulltext_request],
        ranker=ranker,
        limit=5,
        output_fields=["id", "title", "latitude", "longitude", "h3_r8"]
    )

to_dataframe(search_labse_and_pythainlp(query))

,id,distance,title,latitude,longitude,h3_r8
0,ChIJFepMlimf4jARW2MqZCN7GMQ,0.811099,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff
1,ChIJ89mnRNGj4jARXFw7F8kdlu8,0.796627,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
2,ChIJG5cMlYmf4jARjI14Tvhzj5I,0.384878,Sushi Sora,13.726238,100.543182,8864a4b14dfffff
3,ChIJC_IG4WGZ4jARJil71QyJnJQ,0.368889,Min Sushi by Sushi Cottage ずしコテージ,13.740359,100.525108,8864a4b10dfffff
4,ChIJT-MmY-mj4jARBxgvjap0hf0,0.356091,Suki Teenoi Susco Phuttha Bucha,13.651340,100.488991,8864a4b231fffff


🔹 Step 9.6 — Hybrid Search Example: BGE-M3 Dense + Sparse

Combine BGE-M3's dense and sparse outputs with a WeightedRanker to produce hybrid results that favor dense or sparse signals as needed.

In [24]:
# Hybrid search: combine BGE-M3 dense and sparse using client.hybrid_search
from pymilvus import AnnSearchRequest 

def prepare_reqs(query:str):
    # Prepare BGE-M3 embedding for the query
    query_bge_m3_embedding = bge_m3_embedding_func.encode_queries([query])  # dict with "dense" and "sparse"

    # Prepare BGE-M3 dense query
    bge_dense_query = query_bge_m3_embedding["dense"]  # [float, ...]
    bge_dense_request = AnnSearchRequest(
        data=bge_dense_query,  # [float, ...]
        anns_field="bge_m3_dense_vector",
        param={
            "metric_type": "IP",
            "params": {"nprobe": 10}
        },
        limit=10
    )

    # Prepare BGE-M3 sparse query
    bge_sparse_query = query_bge_m3_embedding["sparse"]  # dict
    bge_sparse_request = AnnSearchRequest(
        data=bge_sparse_query,  # sparse_dict
        anns_field="bge_m3_sparse_vector",
        param={
            "metric_type": "IP"
        },
        limit=10
    )
    return [bge_dense_request, bge_sparse_request]

def hybrid_search(reqs, ranker, limit:int=5):
    # Run hybrid search
    return client.hybrid_search(
        collection_name=collection_name,
        reqs=reqs,
        ranker=ranker,
        limit=limit,
        output_fields=["id", "title", "latitude", "longitude", "h3_r8"]
    )


# Request recall: perform searches
reqs = prepare_reqs(query)


In [25]:
from pymilvus import WeightedRanker

# Dense results
to_dataframe(hybrid_search([reqs[0]], WeightedRanker(1.0), limit=10))  

,id,distance,title,latitude,longitude,h3_r8
0,ChIJ89mnRNGj4jARXFw7F8kdlu8,0.674158,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
1,ChIJFepMlimf4jARW2MqZCN7GMQ,0.671508,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff
2,ChIJG5cMlYmf4jARjI14Tvhzj5I,0.664250,Sushi Sora,13.726238,100.543182,8864a4b14dfffff
3,ChIJfR02toKZ4jARBdP-FsD5LHw,0.660441,Yuzu Curry Siam Square Soi.9,13.744181,100.533394,8864a4b16bfffff
4,ChIJE4Fo9SWf4jARBjP_LjHfd2c,0.659965,Fuji Restaurant,13.727249,100.540962,8864a4b14dfffff
5,ChIJ7crcpPaj4jAR3QlRUKokiWI,0.658282,KFC @Susco square,13.650912,100.488785,8864a4b231fffff
6,ChIJNcxj5ZGZ4jAR1QmlhMQwj5g,0.657635,Nijiki,13.734060,100.526360,8864a4b147fffff
7,ChIJT-MmY-mj4jARBxgvjap0hf0,0.655684,Suki Teenoi Susco Phuttha Bucha,13.651340,100.488991,8864a4b231fffff
8,ChIJhSv7MYuf4jAR51cm2YcTwag,0.655371,MK Restaurants,13.744785,100.533951,8864a4b16bfffff
9,ChIJUZp-kVOZ4jAR7ywsoZPWijY,0.654897,Nikaku Bangkok,13.721915,100.528725,8864a4b14bfffff


In [26]:
# Sparse results
to_dataframe(hybrid_search([reqs[1]], WeightedRanker(1.0), limit=10))  

,id,distance,title,latitude,longitude,h3_r8
0,ChIJFepMlimf4jARW2MqZCN7GMQ,0.530283,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff
1,ChIJ89mnRNGj4jARXFw7F8kdlu8,0.519297,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
2,ChIJ7crcpPaj4jAR3QlRUKokiWI,0.500138,KFC @Susco square,13.650912,100.488785,8864a4b231fffff
3,ChIJayNYpeOh4jARD3Aw4pGE0pE,0.500075,Hajime Robot Restaurant Rama 3 (ร้านอาหารฮาจิเ...,13.685045,100.544472,8864a4b301fffff
4,ChIJGVpOLOo72jARSYTubVBMh2E,0.500059,The '90s - Burmese Tea & Noodles Shop,18.797735,98.967598,886482c695fffff
5,ChIJ9d0-4EQx2jARMqOkQAOcCGo,0.500059,หมวย ข้าวซอย Muay Khao Soi ขนมจีนน้ำเงี้ยว อาห...,18.795557,98.965279,886482c691fffff
6,ChIJW0EsNcWr_TARSREAk7a0tvQ,0.500046,Rolling Stone Pizza & Wings HH,12.549917,99.962227,88658bb437fffff
7,ChIJS_B0uzKj4jARyoRkKkXMRo4,0.500043,ERA VALLEY,13.657025,100.468719,8864a4b2e5fffff
8,ChIJQRMf7wOZ4jARQz7dlVrlE48,0.500043,Cozii Steak and Restaurant โคซี่ สเต๊ก,13.721587,100.516533,8864a4b15dfffff
9,ChIJm5ZRHxqj4jARVgsfB1L8ziY,0.500041,ครัวกันเอง,13.651203,100.484299,8864a4b233fffff


In [27]:
# 50% dense, 50% sparse
to_dataframe(hybrid_search(reqs, WeightedRanker(0.5, 0.5), limit=10))  

,id,distance,title,latitude,longitude,h3_r8
0,ChIJFepMlimf4jARW2MqZCN7GMQ,0.600896,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff
1,ChIJ89mnRNGj4jARXFw7F8kdlu8,0.596727,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
2,ChIJ7crcpPaj4jAR3QlRUKokiWI,0.579210,KFC @Susco square,13.650912,100.488785,8864a4b231fffff
3,ChIJG5cMlYmf4jARjI14Tvhzj5I,0.332125,Sushi Sora,13.726238,100.543182,8864a4b14dfffff
4,ChIJfR02toKZ4jARBdP-FsD5LHw,0.330221,Yuzu Curry Siam Square Soi.9,13.744181,100.533394,8864a4b16bfffff
5,ChIJE4Fo9SWf4jARBjP_LjHfd2c,0.329982,Fuji Restaurant,13.727249,100.540962,8864a4b14dfffff
6,ChIJNcxj5ZGZ4jAR1QmlhMQwj5g,0.328817,Nijiki,13.734060,100.526360,8864a4b147fffff
7,ChIJT-MmY-mj4jARBxgvjap0hf0,0.327842,Suki Teenoi Susco Phuttha Bucha,13.651340,100.488991,8864a4b231fffff
8,ChIJhSv7MYuf4jAR51cm2YcTwag,0.327686,MK Restaurants,13.744785,100.533951,8864a4b16bfffff
9,ChIJUZp-kVOZ4jAR7ywsoZPWijY,0.327449,Nikaku Bangkok,13.721915,100.528725,8864a4b14bfffff


In [28]:
from pymilvus import Function, FunctionType

rrf_ranker = Function(
    name="rrf",
    input_field_names=[], # Must be an empty list
    function_type=FunctionType.RERANK,
    params={
        "reranker": "rrf", 
        "k": 100  # Optional
    }
)

# RRF (Reciprocal Rank Fusion) K=100
to_dataframe(hybrid_search(reqs, rrf_ranker, limit=10))  

,id,distance,title,latitude,longitude,h3_r8
0,ChIJ89mnRNGj4jARXFw7F8kdlu8,0.019705,ไข่หวานบ้านซูชิ สาขาประชาอุทิศ,13.660226,100.501335,8864a4b223fffff
1,ChIJFepMlimf4jARW2MqZCN7GMQ,0.019705,sushimai ซูชิมั้ย ศรีบำเพ็ญ,13.721445,100.546730,8864a4b327fffff
2,ChIJ7crcpPaj4jAR3QlRUKokiWI,0.019143,KFC @Susco square,13.650912,100.488785,8864a4b231fffff
3,ChIJG5cMlYmf4jARjI14Tvhzj5I,0.009709,Sushi Sora,13.726238,100.543182,8864a4b14dfffff
4,ChIJayNYpeOh4jARD3Aw4pGE0pE,0.009615,Hajime Robot Restaurant Rama 3 (ร้านอาหารฮาจิเ...,13.685045,100.544472,8864a4b301fffff
5,ChIJfR02toKZ4jARBdP-FsD5LHw,0.009615,Yuzu Curry Siam Square Soi.9,13.744181,100.533394,8864a4b16bfffff
6,ChIJE4Fo9SWf4jARBjP_LjHfd2c,0.009524,Fuji Restaurant,13.727249,100.540962,8864a4b14dfffff
7,ChIJGVpOLOo72jARSYTubVBMh2E,0.009524,The '90s - Burmese Tea & Noodles Shop,18.797735,98.967598,886482c695fffff
8,ChIJ9d0-4EQx2jARMqOkQAOcCGo,0.009434,หมวย ข้าวซอย Muay Khao Soi ขนมจีนน้ำเงี้ยว อาห...,18.795557,98.965279,886482c691fffff
9,ChIJNcxj5ZGZ4jAR1QmlhMQwj5g,0.009346,Nijiki,13.734060,100.526360,8864a4b147fffff
